In [ ]:
import pandas as pd
import numpy as np
import warnings
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

df = pd.read_csv(r'\M5 datasets\sales_train_evaluation.csv')
calendar = pd.read_csv(r'\M5 datasets\calendar.csv')
calendar = calendar.loc[calendar['year']!=2011]
calendar = calendar.loc[calendar['year']!=2016]

df.iloc[:,6:] = df.iloc[:,6:].replace(0, np.nan)
df['nan'] = df.iloc[:,6:].T.isna().sum().T

## Filter data for Foods Category

t = df.loc[df['cat_id']=='FOODS']
days_lst = calendar['d'].unique().tolist()
spd = t[['id','item_id','dept_id','store_id']]
spd [days_lst] = t[days_lst]

# calendar.loc[calendar['month']==1]
calendar.loc[calendar['month']==1, 'q'] = 1
calendar.loc[calendar['month']==2, 'q'] = 1
calendar.loc[calendar['month']==3, 'q'] = 1


calendar.loc[calendar['month']==4, 'q'] = 2
calendar.loc[calendar['month']==5, 'q'] = 2
calendar.loc[calendar['month']==6, 'q'] = 2

calendar.loc[calendar['month']==7, 'q'] = 3
calendar.loc[calendar['month']==8, 'q'] = 3
calendar.loc[calendar['month']==9, 'q'] = 3

calendar.loc[calendar['month']==10, 'q'] = 4
calendar.loc[calendar['month']==11, 'q'] = 4
calendar.loc[calendar['month']==12, 'q'] = 4


spd_main = pd.DataFrame()
i=0
for y in [2012,2013,2014,2015]:
    for q in [1,2,3,4]:
        df_main = pd.DataFrame()
        df_main = spd[['id','item_id','dept_id','store_id']]
        lst = calendar.loc[(calendar['q']==1)&(calendar['year']==y)]['d'].unique().tolist()
        for i in range(0,len(lst)):
            df_main[str(i)] = spd[lst[i]]
        df_main['q'] = q
        df_main['y'] = y
        i=i+1
        if i ==1:
            spd_main = df_main
        else:
            spd_main = pd.concat([spd_main, df_main], axis = 0)
        
 
# Filter 44% Nan
spd_main['nan'] = spd_main.iloc[:, 4:-2].T.isna().sum().T
spd_main = spd_main.loc[spd_main['nan']<=40]
del spd_main['nan']

# TS
TS = spd_main.copy()
TS.to_csv(r'TS.csv', index = False)

Fuzzification

In [ ]:
spd_main.iloc[:, 4:-2] = spd_main.iloc[:, 4:-2].fillna(0)

# Define alpha1 function
def alpha1(alphacu, kcu):
    alphamoi = alphacu / (1 + alphacu * kcu)
    return alphamoi.T


class FuzzificationProcessor:
    def __init__(self):
        self.i = 0
        self.data = []
        self.rmse = []
    
    def fuzzification(self, row_array):
       
        W = row_array
        zt = (W - np.min(W)) / (np.max(W) - np.min(W))
        l = 12

        # Initial distance and mean distance calculation
        d = squareform(pdist(zt.reshape(-1, 1), 'euclidean'))
        d1 = np.triu(d, 1)
        cn = (zt.size * (zt.size - 1)) / 2
        mu = np.sum(d1) / cn

        # Standard deviation calculation
        tong = (d - mu) ** 2
        tt = np.triu(tong, 1)
        sigma = np.sqrt(np.sum(tt) / cn)

        # Weight matrix calculation
        f = np.exp(-d / (sigma / l)) * (d <= mu)
        np.fill_diagonal(f, 0)

        # Iterative update of ztmoi
        iter_count = 0
        exilanh = 0.01

        j = 0
        while True:
            tu = np.dot(f, zt)
            mau = np.sum(f, axis=1)
            mau = np.where(mau == 0, 0.00000000001, mau)
            ztmoi = tu / mau
            
            if np.max(np.abs(ztmoi - zt)) <= exilanh:
                break
            zt = ztmoi
            iter_count += 1
            d = squareform(pdist(zt.reshape(-1, 1), 'euclidean'))
            d1 = np.triu(d, 1)
            mu = np.sum(d1) / cn
            tong = (d - mu) ** 2
            tt = np.triu(tong, 1)
            sigma = np.sqrt(np.sum(tt) / cn)
            alp = alpha1(np.ones((zt.size, zt.size)), f)
            f = np.exp(-d / (sigma / l)) * (d <= mu)
            np.fill_diagonal(f, 0)
            j=j+1
            if j>=2000:   # For faster runtime
                break



    # Calculate U_new
        # epsilon = 1e-2
        
        
        # center = np.unique(np.round(ztmoi, 2))
        
        # dist = squareform(pdist(center.reshape(-1, 1)))
        # dist[dist == 0] = epsilon
        # tmp = dist ** (-2 / (2 - 1))
        # U_new = tmp / np.sum(tmp, axis=0)


   
        # self.data.append({
        #     'indx' : self.i,
        #     'min' : np.min(W),
        #     'max' : np.max(W),
        #     'Umatrix' : U_new,
        #     }
        #     )
        # self.i += 1
        # print(np.round(self.i/740, 2))
        FTS = zt* (np.max(W) - np.min(W)) + np.min(W)
        FTS = np.round(FTS, 1)

        if np.any(np.isnan(FTS)):
            FTS = W.copy()
        self.rmse.append(np.sqrt((abs(W-FTS))**2).mean())
        return FTS

processor = FuzzificationProcessor()

spd_main.iloc[:, 4:-2] = spd_main.iloc[:, 4:-2].apply(lambda row: pd.Series(processor.fuzzification(row.to_numpy())), axis=1)

# processed_data = processor.data

spd_main.to_csv(r'FTS.csv', index = False)

model

In [ ]:
def correlation(df, threshold):
        numeric_df = df.select_dtypes(include=['number'])
        
        col_corr = set()
        corr_matrix = numeric_df.corr()
        
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold:
                    colname = corr_matrix.columns[i]
                    col_corr.add(colname)
        
        return col_corr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import  train_test_split
def train(trget, real_df, label, year):
    
    
    df = trget.copy()
    df['id'] = df['id'].astype('category') 
    df['item_id'] = df['item_id'].astype('category') 
    df['dept_id'] = df['dept_id'].astype('category') 
    df['store_id'] = df['store_id'].astype('category') 
    X = df.copy()
    
    ################
    real_df_y = real_df.loc[real_df['y']==2015][str(label)]

    eval_y = real_df_y

    #########################
    y = X.loc[X['y']<year][str(label)]
    # eval_y = eval_X[str(label)]

    for l in range(label, 91):
          del X[str(l)]
          

    #####################################################Feature Extraction
    last_y = label - 1

    icols =  [['id'],
                ['item_id'],
                ['dept_id'],
                ['store_id']]
    for d in [last_y-1, last_y-2, last_y-3, last_y-4, last_y-5]:
            for col in icols:
                    col_name = '_' + '_'.join(col) + '_'
                    X['enc' + col_name +str(last_y - d)+ '_mean'] = X.groupby(col)[str(d)].transform('mean')
                    X['enc' + col_name + str(last_y - d)+ '_median'] = X.groupby(col)[str(d)].transform('median')
                    X['enc' + col_name +str(last_y - d)+ '_std']    = X.groupby(col)[str(d)].transform('std')

                    X['enc' + col_name + str(last_y - d)+ '_mean'] = X.groupby(col)['enc' + col_name +str(last_y - d)+ '_mean'].transform(lambda x: x.fillna(method='bfill').fillna(method='ffill'))
                    X['enc' + col_name + str(last_y - d)+ '_median'] = X.groupby(col)['enc' + col_name + str(last_y - d)+ '_median'].transform(lambda x: x.fillna(method='bfill').fillna(method='ffill'))
                    X['enc' + col_name +str(last_y - d)+ '_std'] = X.groupby(col)['enc' + col_name + str(last_y - d)+ '_std'] .transform(lambda x: x.fillna(method='bfill').fillna(method='ffill'))

    for i in [2, 3, 7, 14, 30]:
        r = pd.DataFrame()
        for j in range(0,i):
            r[str(j)] = X[str(last_y-j)]
            
        X['rolling_mean_' + str(i)]   = r.mean(axis=1)
        X['rolling_median_' + str(i)] = r.median(axis=1)
        X['rolling_std_' + str(i)]    = r.std(axis=1)
        X['rolling_max_' + str(i)]    = r.max(axis=1)

    

    cor_feature = correlation(X, 0.97)
    features_columns = X.columns.tolist()
    features_columns = [x for x in features_columns if x not in cor_feature]

    eval_X = X.loc[X['y']==2015]
    X = X.loc[X['y']<year]
    X = X.drop(columns = cor_feature)
    
    eval_X = eval_X.drop(columns = cor_feature)
    print('delete this cols for corr:', cor_feature)
    
    print(X.shape)
    print(y.shape)
    print(eval_X.shape)
    print(eval_y.shape)
    #######################################################################

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30,shuffle=True)


    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',          
        'tweedie_variance_power': 1.2,      
        'metric': 'rmse',                   
        'subsample': 0.5,                 
        'subsample_freq': 1,             
        'learning_rate': 0.005,          
    #     'max_depth' : 12,                 
    #     'early_stopping_round' : 1,         
        'num_leaves': 2**12-1,
        'min_data_in_leaf': 2**8-1,           
        'feature_fraction': 0.5,           
        'max_bin': 100,                     
        'n_estimators': 15000,               
        'boost_from_average': False,        
        'verbose': -1,                      
        'SEED': 42
    }
    le_callback = lgb.log_evaluation(5)
    es_callback = lgb.early_stopping(5)

    train_data = lgb.Dataset(X_train,
                            label = y_train)
    valid_data = lgb.Dataset(X_test,
                            label = y_test)
    
    evals_result = {}

    estimator = lgb.train(
            params,
            train_set=train_data,
            valid_sets=[train_data, valid_data],
            valid_names=['train', 'valid'],
            callbacks=[lgb.record_evaluation(evals_result),le_callback, es_callback],
            
        )
    epochs = len(evals_result['train']['rmse'])
    x_axis = range(0, epochs)

    plt.figure(figsize=(10, 5))
    plt.plot(x_axis, evals_result['train']['rmse'], label='Train Loss')
    plt.plot(x_axis, evals_result['valid']['rmse'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE Loss')
    plt.title('LightGBM RMSE Loss'+', cv:'+str(year - 1))
    plt.legend()
    plt.show()
    FEA = pd.DataFrame({'name':estimator.feature_name(),
                            'importance_gain': estimator.feature_importance(importance_type='gain'),
                            #'importance_split': estimator.feature_importance(importance_type='split'),
                            }).sort_values('importance_gain', ascending=False)
    display(FEA.head(30))


    y_hat = estimator.predict(eval_X)
    eval_df = pd.DataFrame()
    eval_y=eval_y.reset_index(drop=True)
    eval_X=eval_X.reset_index(drop=True)
    y_hat = estimator.predict(eval_X)
    eval_df['pred']=y_hat
    eval_df['real'] = eval_y
    eval_df['MAPE'] = abs(eval_df['pred'] - eval_df['real'])/eval_df['real']
    eval_df['squar_error'] = (abs(eval_df['pred'] - eval_df['real']))**2

    return eval_df

model for TS

In [ ]:
cv = 2
for c in range(1, cv + 1):
    year = 2015 - c
    print('cv '+str(c)+':')
    ts_result = train(TS, TS, 89, year)
    print(ts_result)
    print('RMSE without zeros in real : ', ts_result.loc[ts_result['real']!=0]['squar_error'].mean())
    print("MAPE without zeros in real : ",ts_result.loc[ts_result['real']!=0]['APE'].mean())
    print('RMSE : ', ts_result['squar_error'].mean())

In [ ]:
cv = 2
for c in range(1, cv + 1):
    year = 2015 - c
    print('cv '+str(c)+':')
    fts_result = train(FTS, TS, 89, year)
    print(fts_result)
    print('RMSE without zeros in real : ', fts_result.loc[fts_result['real']!=0]['squar_error'].mean())
    print("MAPE without zeros in real : ",fts_result.loc[fts_result['real']!=0]['APE'].mean())
    print('RMSE : ', fts_result['squar_error'].mean())